## DeepSEE: Deep Disentangled Semantic Explorative Extreme Super-Resolution

This is a demo for our paper **Deep Disentangled Semantic Explorative Extreme Super-Resolution** ([ACCV 2020](http://accv2020.kyoto/) oral). 
Please check out our [project page](https://mcbuehler.github.io/DeepSEE/) for details and the paper.

#### Setup
1. Download and unpack the model checkpoints.
2. Download and unpack the demo data.
3. Install the requirements.

Please refer to the [README](README.md) for instructions.

In [4]:
import os
from demo import Demo, get_demo_options, display_result
import torch
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

%load_ext autoreload
%autoreload 2
opt = get_demo_options("32x_independent")
demo = Demo(opt)
base_path = "demo_data/"
dataset = "CelebAMask-HQ"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Inference for the __Default__ Solution
We upscale a low-resolution (16x16) image to a single high-resolution variant for 32x upscaling. We call this the __default solution__.

We provide these sample images from the [CelebAMask-HQ](https://github.com/switchablenorms/CelebAMask-HQ) dataset (bicubically downscaled to 16x16 pixels):

In [5]:
os.listdir(os.path.join(base_path, dataset, "image_16x16"))

['14337.png', '19776.png', '28368.png']

For the default solution, we need to provide two inputs:
* the low-resolution input and 
* its predicted semantic mask (pre-computed).

In [6]:
filename = "14337.png"

kwargs = {
    "name": "demo",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "predicted_labels", filename)
}
result_default_solution = demo.run(**kwargs)
display_result(result_default_solution)

Encoding style from LR image...
Style computed.
Upscaling...
Done.


/home/buehlmar/data/software/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…

### Semantic Manipulations
We can repaint the semantic mask and run inference again. This yields different shapes in the upscaled image, but preserves the overall appearance.

In [7]:
kwargs = {
    "name": "demo_semantic_manipulation",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "manipulated_labels", filename)
}
result_default_solution = demo.run(**kwargs)
display_result(result_default_solution)

Encoding style from LR image...
Style computed.
Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…

### Style Manipulations
We can add random noise to generate multiple high-resolution variants. In the following example, we change the smooth skin texture by adding noise to the corresponding row in the style matrix.

* Change `delta` to make the effect stronger or weaker.
* You can choose to influence other/multiple semantic regions by changing/adding indices to `region`.
* The style matrix (or rows thereof) can be computed from other high-resolution images (examples in the paper).
* Rows in the style matrix can be interpolated (examples in the paper).

In [8]:
encoded_style_orig = result_default_solution["encoded_style"].detach().clone()
encoded_style_noisy = encoded_style_orig.clone()
import torch
delta = 0.15
region = [1]  # Corresponds to the skin region.

encoded_style_noisy[:, region, :] = (encoded_style_orig[:, region, :] + (torch.rand(encoded_style_orig[:, region, :].shape) * delta)).clamp(-1, 1)
kwargs = {
    "name": "demo_style_manipulation",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "manipulated_labels", filename),
    "encoded_style": encoded_style_noisy
}
result = demo.run(**kwargs)
display_result(result)

Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…